In [1]:
from SALib.analyze import sobol
from SALib.sample import saltelli
from SALib.test_functions import Ishigami
import numpy as np
from math import *
from threading import Thread
from time import time
from numba import jit
from numba import njit

In [2]:
def evaluate_model(x):
    return x[0]*x[0] + 10*sin(x[1]) + 2*cos(x[2])


In [3]:
@jit(nopython=True)
def evaluate_model_jit(x):
    return x[0]*x[0] + 10*sin(x[1]) + 2*cos(x[2])

In [4]:
problem = {
    'num_vars' : 3,
    'names' : ['x1', 'x2', 'x3'],
    'bounds' : [[-10,10],
                [-10,10],
                [-10,10]
               ]
}

In [6]:
# Чистый вариант с которым будем сравнивать
for n in [10000,100000,500000]:
    print(f"n ={n}")
    start = time()
    param_values = saltelli.sample(problem, n)
    Y = np.zeros(param_values.shape[0])
    print("samples generation took %s seconds" %(time() - start))

    
    start = time()
    for i, X in enumerate(param_values):
        Y[i] = evaluate_model(X)
    print("model evaluation took %s seconds" %(time() - start))

    start = time()
    S = sobol.analyze(problem, Y)
    print("SA took %s seconds" %(time() - start))

n = 10000
samples generation took 0.4762837886810303 seconds
model evaluation took 0.17675018310546875 seconds
SA took 0.5971724987030029 seconds
n = 100000
samples generation took 4.266716003417969 seconds
model evaluation took 1.7974331378936768 seconds
SA took 9.750931978225708 seconds
n = 500000
samples generation took 22.345986366271973 seconds
model evaluation took 9.009239673614502 seconds
SA took 64.34293127059937 seconds


In [7]:
S['S1']

array([0.94695349, 0.05083665, 0.00221696])

In [8]:
S['S2']

array([[            nan,  5.39452631e-06,  4.69504307e-06],
       [            nan,             nan, -5.06544027e-06],
       [            nan,             nan,             nan]])

In [9]:
#попробуем подключить Numba

for n in [10000,100000,500000]:
    print(f"n ={n}")
    start = time()
    param_values = saltelli.sample(problem, n)
    Y = np.zeros(param_values.shape[0])
    print("samples generation took %s seconds" %(time() - start))


    start = time()
    for i, X in enumerate(param_values):
        Y[i] = evaluate_model_jit(X)
    print("model evaluation took %s seconds" %(time() - start))

    start = time()
    S = sobol.analyze(problem, Y)
    print("SA took %s seconds" %(time() - start))

n = 10000
samples generation took 0.518578290939331 seconds
model evaluation took 1.1278417110443115 seconds
SA took 0.5360112190246582 seconds
n = 100000
samples generation took 4.1586644649505615 seconds
model evaluation took 0.7711987495422363 seconds
SA took 8.106122016906738 seconds
n = 500000
samples generation took 21.140419006347656 seconds
model evaluation took 3.7271618843078613 seconds
SA took 52.76559495925903 seconds


(!): буст скорости для evaluate_model х3-x5

In [5]:
from threading import Thread

In [21]:
#попробуем подключить мультипоточность на 4 потока
num_threads=4
n=500000

def evaluate_model_parallel(thread_id):
    for i, X in enumerate(param_values_lst[thread_id]):
        Y_lst[thread_id][i] = evaluate_model(X)
    
print("n =",str(n))
start = time()
param_values = saltelli.sample(problem, n)
size_for_thread = param_values.shape[0]//num_threads
param_values_lst = []
Y_lst = []
for i in range(num_threads):
    first = i*size_for_thread
    last = (i+1)*size_for_thread
    param_values_lst.append(param_values[first:last,])
    Y_lst.append(np.zeros(param_values_lst[i].shape[0]))
print("samples generation took %s seconds" %(time() - start))


start = time()
threads_list=[]
for thread_id in range(num_threads):
    thread=Thread(target=evaluate_model_parallel, args=(thread_id,))
    threads_list.append(thread)
    thread.start()      
for thread in threads_list:
    thread.join()                   
Y = np.concatenate((Y_lst),axis=0)                  
print("model evaluation took %s seconds" %(time() - start))


start = time()
S = sobol.analyze(problem, Y)
print("SA took %s seconds" %(time() - start))

n = 500000
samples generation took 21.59383535385132 seconds
model evaluation took 8.506713390350342 seconds
SA took 63.58519005775452 seconds


In [22]:
#проверим, что результат правильный и совпадает начальным вариантом
S['S1']

array([0.94695349, 0.05083665, 0.00221696])